# Preparing Document for Data generation
- This notebook will show you how to do document parsing
- Document Chunking
- And finally mixing it with user QNA to  create seed examples

### Install SDG

```bash 
pip install sdg-hub==0.1.0a2
pip install rich datasets tabulate transformers
```

### Create Seed Examples

In [8]:
from sdg_hub.utils.docprocessor import DocProcessor

output_dir = f"sdg_demo_output/"
# This is where your PDFs are stored
data_dir = 'wikipedia/' 
# It also have your QNA yaml file
dp = DocProcessor(data_dir, user_config_path=f'{data_dir}/qna.yaml')

### Using docling v1 json
seed_data = dp.get_processed_dataset()

### Using markdown file
seed_data = dp.get_processed_markdown_dataset([f"{data_dir}/document_1.md"])

# Note: For now v2 json is not supported

seed_data.to_json(f'{output_dir}/seed_data.jsonl', orient='records', lines=True)

Creating json from Arrow format: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 397.94ba/s]


10983